# 基本的模組

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import preprocessing
import time
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 指定只是用第1塊GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


# 資料處裡

In [3]:
def Fileloader(PreData, LoaderName):
    PurName, PreName="", ""
    
    PurName += LoaderName

    if  (PreData==0): PreName += '00Raw'
    elif(PreData==1): PreName += '01Smo'
    elif(PreData==2): PreName += '02Smo_Diff'
    elif(PreData==3): PreName += '03Smo_SNV_Diff'
    elif(PreData==4): PreName += '04Smo_MSC_Diff'
    elif(PreData==5): PreName += '05Smo_SNV'
    elif(PreData==6): PreName += '06Smo_MSC'
        
    Method_Name = PurName + PreName
    all_df_Origin = pd.read_excel(os.getcwd()+"/DataBase/" + Method_Name + ".xlsx")
    Method_Name2= PurName + "," + PreName
    return all_df_Origin, Method_Name2

def LabelToInt(DataTable):
    df=DataTable
    df=df.drop(['No', 'Name', 'Time', 'Country', 'Area', 'Variety_0', 'Treatment', 'Baking', 'Flavor', 'Gram', 'Status'], axis=1)
    print(np.unique(df['Variety']))
    df['Variety']= df['Variety'].map({'阿拉比卡':0, '阿拉比卡&羅布斯塔':1, '羅布斯塔':2}).astype(int)
    return df

#Label & Features 切割
def PartData(df):
    ndarray = df.values   #取得數據轉值
    Label = ndarray[:,0]     #取得Label資料
    Features = ndarray[:,1:]  #取得Features資料
    return Features,Label
#資料比例設定
def PartTable(DF):
    Flag_Variety=np.unique(DF['Variety']) #確認要辨識的Label
    print(Flag_Variety)
    DV0, DV1, DV2=DF[(DF.Variety == Flag_Variety[0])], DF[(DF.Variety == Flag_Variety[1])], DF[(DF.Variety == Flag_Variety[2])]
    #選取該Label的列表
    Flag_0_Name, Flag_1_Name, Flag_2_Name=np.unique(DV0['Name']), np.unique(DV1['Name']), np.unique(DV2['Name'])
    #3個Label各自對應的Name
    cho_Flag_0 = np.random.choice(Flag_0_Name)          #Label=0 (羅布斯塔) 隨機選1個
    cho_Flag_1 = np.random.choice(Flag_1_Name, size=3)  #Label=1 (阿拉比卡) 隨機選3個
    cho_Flag_2 = np.random.choice(Flag_2_Name)          #Label=2 (混和咖啡) 隨機選1個(只有1個)
    
    mskTrain=((DF.Name == cho_Flag_0)| 
              (DF.Name == cho_Flag_1[0])| (DF.Name == cho_Flag_1[1])| (DF.Name == cho_Flag_1[2])|
              (DF.Name == cho_Flag_2))
    Train = DF[~mskTrain]
    
    mskTest=((DF.Name == cho_Flag_0)| 
             (DF.Name == cho_Flag_1[0])| (DF.Name == cho_Flag_1[1])| (DF.Name == cho_Flag_1[2]))
    Test = DF[mskTest]
    
    mskException=(DF.Name == cho_Flag_2)
    Exception = DF[mskException]
    
    return Train, Test, Exception

In [4]:
import math
def CellSetting(InCell, OutCell):
    #輸入 特徵欄位數(輸入神經元), 標籤欄位數(輸入神經元)
    #輸出 神經元1, 神經元2, 神經元3
    D1 = round(2 * math.sqrt((OutCell+2)*InCell))
    D2 = round(math.sqrt((OutCell+2)*InCell)+2*math.sqrt(InCell/(OutCell+2)))
    D3 = round(OutCell * math.sqrt(InCell/(OutCell+2)))
    return D1,D2,D3

# 模型的設定

In [5]:
from keras.models import Sequential
from keras.layers import Dense,Dropout, Input
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import accuracy_score

In [6]:
def MLPModelCreate(InCell, D1, D2, D3, OutCell):
    model = Sequential()
    model.add(Input(shape=(InCell,)))  # 確保這裡是元組形式
    model.add(Dense(units=D1,     kernel_initializer='uniform', activation='relu'))
    model.add(Dense(units=D2,     kernel_initializer='uniform', activation='relu'))
    model.add(Dense(units=D3,     kernel_initializer='uniform', activation='relu'))
    model.add(Dense(units=OutCell,kernel_initializer='uniform', activation='sigmoid'))
    #OutCell為Label欄位數量
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# 模型訓練
def ModelTrain(model, train_Features, train_Label):
    train_history = model.fit(x=train_Features, y=train_Label, validation_split=0.1, epochs=16, batch_size=1,verbose=0, shuffle=True)
    print("==========Train End==========")
    return model, train_history

# 模型的預測結果 & 機率
def ModelPrediction(model, Features):
    prediction=np.argmax(model.predict(Features), axis=-1)
    Predicted_Probability=model.predict(Features)
    return prediction, Predicted_Probability

In [7]:
def Main(PreData, LoaderName):
    DataTable_0, Method_Name = Fileloader(PreData, LoaderName)
    print('資料集設定：' + Method_Name)
    #============================================= 
    DfTrain, DfTest, DfException=PartTable(DataTable_0)
    #============================================= 
    DataTable_Train = LabelToInt(DfTrain)
    DataTable_Test = LabelToInt(DfTest)
    DataTable_Exception = LabelToInt(DfException)
    #=============================================
    train_Features, train_Label = PartData(DataTable_Train)
    test_Features,  test_Label  = PartData(DataTable_Test)
    Exception_Features,  Exception_Label  = PartData(DataTable_Exception)

    train_Label, test_Label = pd.DataFrame(train_Label), pd.DataFrame(test_Label) 
    train_Label, test_Label = train_Label.align(test_Label, join='outer',fill_value = 0, axis=1)
    train_Label, test_Label = train_Label.values, test_Label.values
    #=============================================
    train_Features, test_Features, Exception_Features = train_Features[:,:], test_Features[:,:], Exception_Features[:,:]
    
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1)) #設定資料範圍
    minmax_scale.fit(train_Features)
    train_Features=minmax_scale.transform(train_Features)       #將Features數值轉換至0~1之間
    test_Features=minmax_scale.transform(test_Features)         #將Features數值轉換至0~1之間
    Exception_Features = minmax_scale.transform(Exception_Features)    
    #=============================================
    InCell, OutCell = len(train_Features[0]), len(train_Label[0])
    D1, D2, D3 = CellSetting(InCell, OutCell)

    Model_Name="MLP"
    print('模型設定：' + Model_Name)

    Model1 = MLPModelCreate(InCell, D1, D2, D3, OutCell)
    Model2, Train_History= ModelTrain(Model1, train_Features, train_Label)
    #=============================================
    prediction, Predicted_Probability = ModelPrediction(Model2, test_Features)
    
    Test_Accuracy=accuracy_score(np.argmax(test_Label, axis=1), prediction)
    print('準確率: %.3f' % Test_Accuracy)
    scores = Model2.evaluate(x=test_Features, y=test_Label, verbose=1)
    print('誤差值(test): %.3f' % scores[0], '評估標準值(test): %.3f' % scores[1])
    #=============================================
    Exception_prediction, Exception_Probability = ModelPrediction(Model2, Exception_Features)
    
    #=============================================
    #Model2.save(os.getcwd()+"/Model/" + Model_Name + '_' + Method_Name+'.h5')
    #print("Saved model to disk")
    #=============================================

In [ ]:
Main(1, '咖啡粉')